# Data Preparing

In [1]:
import pandas as pd
data = pd.read_parquet('data_with_categories.parquet').reset_index(drop = True)
data

,product_id,category_id,sale,shop_id,shop_title,rating,text_fields,category_name,cat_1,cat_2
0,325286,12171,False,9031,Aksik,5.00000,"{""title"": ""Зарядный кабель Borofone BX1 Lightn...",Аксессуары и запчасти->Зарядные устройства и к...,Электроника,Смартфоны и телефоны
1,888134,14233,False,18305,Sela,5.00000,"{""title"": ""Трусы Sela"", ""description"": ""Трусы-...",Белье и купальники->Трусы,Одежда,Женская одежда
2,1267173,13429,False,16357,ЮНЛАНДИЯ канцтовары,5.00000,"{""title"": ""Гуашь \""ЮНЫЙ ВОЛШЕБНИК\"", 12 цветов...","Краски, пигменты",Хобби и творчество,Рисование
3,1416943,2789,False,34666,вася-nicotine,4.00000,"{""title"": ""Колба для кальяна Крафт (разные цве...",Кальяны и аксессуары->Колбы,Хобби и творчество,Товары для курения
4,1058275,12834,False,26389,Lim Market,4.60000,"{""title"": ""Пижама женская, однотонная с шортам...",Домашняя одежда->Пижамы,Одежда,Женская одежда
...,...,...,...,...,...,...,...,...,...,...
91115,114402,14922,False,4955,СТЕКЛОФФ ПРО,3.62069,"{""title"": ""Прочное стекло 2D на Samsung Galaxy...",Аксессуары и запчасти->Защитные стекла и пленк...,Электроника,Смартфоны и телефоны
91116,1594500,13028,False,19626,Hobby room,5.00000,"{""title"": ""Алмазная мозаика \""Ромашки\"" 40*50с...",Алмазные мозаики,Хобби и творчество,"Пазлы, мозаика и фреска"
91117,790493,13407,False,22291,Море открыток,5.00000,"{""title"": ""Открытка \""Вместе навсегда\"" в краф...",Открытки и конверты->Открытки,Товары для дома,Товары для праздников
91118,114509,12100,False,2985,Oppa Market,5.00000,"{""title"": ""Пульт K10B-C1 для Rolsen"", ""descrip...",Оборудование для телевизоров->Пульты ДУ,Электроника,Телевизоры и видеотехника


In [2]:
import json
def parse_desc(text):
    titles = []
    desc = []
    for i, doc in enumerate(text):
        titles.append(json.loads(doc)['title'])
        desc.append(json.loads(doc)['description'])
    return titles, desc

In [3]:
data['title'], data['description'] = parse_desc(list(data['text_fields']))
data.drop(columns=['text_fields'], inplace = True)

In [4]:
data = data.drop(columns=[
                   'category_name',
                   'rating',
                   'shop_title',
                   'sale',
                   'product_id'])

In [5]:
def clean_description(corpus): # функция для очистки значений как <что-то>
    
    to_output = True
    out_str = ''
    output = []
    
    for text in corpus:
        for ch in text:
            
            if ch == '<':
                to_output = False
                
            if ch == '>':
                to_output = True
                continue
                
            if to_output:
                out_str += ch
                
        output.append(out_str)
        out_str = ''
    
    return output
data['description'] = clean_description(list(data['description']))

In [6]:
data['cat_1'], _ = data['cat_1'].factorize()

# Lemmatize

In [7]:
import nltk
from nltk import word_tokenize
nltk.download("punkt")
nltk.download('stopwords')
from nltk.corpus import stopwords 
from nltk.stem import SnowballStemmer
from sklearn.pipeline import Pipeline
import string
import numpy as np
import pymorphy2

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vnvof\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vnvof\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
ru_sw = stopwords.words("russian") # ru stop words
snowball = SnowballStemmer(language = "russian")# stemming
morph = pymorphy2.MorphAnalyzer()

def token_all_proccesing(df: str, remove_stop_words: bool = True): # func
    output = ''
    tokens = word_tokenize(df, language="russian") # tokenize
    tokens = [i for i in tokens if i not in string.punctuation] # tokenize w/o punctuation
    if remove_stop_words:
        tokens = [i for i in tokens if i not in ru_sw]# remove ru stopword
    #tokens = [snowball.stem(i) for i in tokens] # snowball stemming
    tokens = [morph.parse(i)[0].normal_form for i in tokens] # lemmatization
    
    for i in tokens:
        output += i + ' '
    
    return output

In [9]:
data

,category_id,shop_id,cat_1,cat_2,title,description
0,12171,9031,0,Смартфоны и телефоны,Зарядный кабель Borofone BX1 Lightning для айф...,Зарядный кабель Borofone BX1 подходит для заря...
1,14233,18305,1,Женская одежда,Трусы Sela,Трусы-слипы из эластичного бесшовного трикотаж...
2,13429,16357,2,Рисование,"Гуашь ""ЮНЫЙ ВОЛШЕБНИК"", 12 цветов по 35 мл, БО...",Гуашь высшего качества ЮНЛАНДИЯ поможет создат...
3,2789,34666,2,Товары для курения,Колба для кальяна Крафт (разные цвета),Универсальная колба для кальяна Craft подходит...
4,12834,26389,1,Женская одежда,"Пижама женская, однотонная с шортами",Лёгкая ткань! Комфортная посадка! Идеальная дл...
...,...,...,...,...,...,...
91115,14922,4955,0,Смартфоны и телефоны,Прочное стекло 2D на Samsung Galaxy J5 Prime,Защитное стекло 2D для Samsung Galaxy J5 Prime...
91116,13028,19626,2,"Пазлы, мозаика и фреска","Алмазная мозаика ""Ромашки"" 40*50см на подрамнике",Алмазная мозаика - очень увлекательное совреме...
91117,13407,22291,3,Товары для праздников,"Открытка ""Вместе навсегда"" в крафтовом конверт...",Открытка 10*15 в крафтовом конверте.
91118,12100,2985,0,Телевизоры и видеотехника,Пульт K10B-C1 для Rolsen,Подходит к аппаратуре Ролсен:Rolsen телевизор ...


In [10]:
data.description = data.description.apply(func = lambda x: token_all_proccesing((x)))

In [11]:
cat_0_df = data[data['cat_1'] == 0]
cat_1_df = data[data['cat_1'] == 1]
cat_2_df = data[data['cat_1'] == 2]
cat_3_df = data[data['cat_1'] == 3]
cat_4_df = data[data['cat_1'] == 4]

In [12]:
cat_3_df

,category_id,shop_id,cat_1,cat_2,title,description
13,15035,11916,3,Декор и интерьер,"Сувенирные деньги, банк приколов",юмор – настоящий богатство купюра пачка выгляд...
20,13027,4827,3,Товары для кухни,Чайник заварочный 0.5 л с металлическим ситечком,чайник заварочный 0.5 л металлический ситечко ...
30,12301,11973,3,Товары для кухни,"Кружка с принтом, в подарочной упаковке",дать серия кружка идеально подходить подарок б...
33,12604,44999,3,Товары для праздников,"Воздушный шар ""Цифра 6"" 80 см, золотой",– красивый воздушный шар фольгировать алюминий...
39,14969,34483,3,Декор и интерьер,"Фоторамка-коллаж, мультирамка ""Love&Family"", 8...",фоторамка-коллаж помочь украсить интерьер ярки...
...,...,...,...,...,...,...
91105,13545,12653,3,Хранение вещей,Рифленая пленка рулон пакеты для вакуумного у...,подходить приготовление су-вид заморозок мороз...
91107,12301,1430,3,Товары для кухни,Кружка для кофе 475 мл/Стакан для напитков/Бок...,о товарепосуд стекло двойной стенка – это гара...
91108,14003,37701,3,Декор и интерьер,"Диффузор ароматический с палочками, аромадиффузор",ароматический диффузор это самый простой изящн...
91114,13104,53732,3,Товары для кухни,Пельменница алюминиевая,пельменница быстрый лепка пельменей.форм пельм...


In [13]:
cat_0_df.to_parquet('big_categorical_parsed_data/cat_0.parquet')
cat_1_df.to_parquet('big_categorical_parsed_data/cat_1.parquet')
cat_2_df.to_parquet('big_categorical_parsed_data/cat_2.parquet')
cat_3_df.to_parquet('big_categorical_parsed_data/cat_3.parquet')
cat_4_df.to_parquet('big_categorical_parsed_data/cat_4.parquet')